In [1]:
import requests
from bs4 import BeautifulSoup
from urllib.request import urlparse, urljoin
from tqdm.auto import tqdm
import pandas as pd
import shutup
shutup.please()

In [2]:
internal_urls = set() # links of the website
external_urls = set() # links of other website
bad_urls = set() # not properly formatted links
error_urls = set() # links with connection errors
total_urls_visited = 0 #  counter for links visited by crawler
max_urls=3 # maximum number of links to be visited by crawler
ConnectErrs  = (ConnectionError, OSError, ConnectionResetError) # list of errors to be catched
scrapped_home_data = pd.DataFrame() #dataframe to hold all scrapped data for home products
scrapped_business_data = pd.DataFrame() #dataframe to hold all scrapped data for business products

In [3]:
def is_valid(url):
    """
    Checks whether `url` is a valid URL.
    """
    if url.startswith('mailto') or url.startswith('tel') or url.startswith('/brand'):
        bad_urls .add(url)
        return False
    parsed = urlparse(url)
    return bool(parsed.netloc) and bool(parsed.scheme)

In [4]:
def get_all_website_links(url):
    """
    Returns all URLs that is found on `url` in which it belongs to the same website
    """
    # all URLs of `url`
    urls = set()
    # domain name of the URL without the protocol
    domain_name = urlparse(url).netloc
    soup = BeautifulSoup(requests.get(url).content, "html.parser")
    for a_tag in soup.findAll("a"):
        href = a_tag.attrs.get("href")
        if href == "" or href is None:
            # href empty tag
            continue
          # join the URL if it's relative (not absolute link)
        href = urljoin(url, href)
        parsed_href = urlparse(href)
        # remove URL GET parameters, URL fragments, etc.
        href = parsed_href.scheme + "://" + parsed_href.netloc + parsed_href.path
        if not is_valid(href):
            # not a valid URL
            continue
        if href in internal_urls:
            # already in the set
            continue
        if domain_name not in href:
            # external link
            if href not in external_urls:
                external_urls.add(href)
            continue
        urls.add(href)
        internal_urls.add(href)
    return urls

In [5]:
def crawl(url):
    """
    Crawls a web page and extracts all links.
    You'll find all links in `external_urls` and `internal_urls` global set variables.
    links with connections issues are in 'error_urls'
    """
    global total_urls_visited
    global max_urls
    global cpbar
    total_urls_visited += 1
    cpbar.update(1)
    links = get_all_website_links(url)
    for link in links:
        if total_urls_visited >= max_urls:
            cpbar.close()
            break
        try:
            crawl(link)
        except ConnectErrs as e:
            error_urls.add(url)
            continue

In [6]:
def scrap_home(link):
    global scrapped_home_data
    page = requests.get(link)
    soup = BeautifulSoup(page.content, "html.parser")
    html = list(soup.children)[1]
    body = list(html.children)[1]
    if len(str(list(body.children)[-1])) > 48: # check if data available
        item_dict = eval(str(list(body.children)[-1])[35:-9].replace('\n','').replace('@','').replace('[','').replace(']',''))
        df = pd.json_normalize(item_dict, record_prefix='Prefix.') # to be used with single record
        image_element = soup.find_all("img")[2]['src'] # scrap the image link
        df['image'] = image_element # adding the image link to results
        page_text = soup.get_text('/n') # add seperator so you can split later
        relevant_beginning = page_text.find('Main') + 15
        relevant_ending = page_text.find('addGeneral Terms')
        product_and_related = page_text[relevant_beginning:relevant_ending]
        details_beginning = product_and_related.find('Details') + 7
        details_ending  = product_and_related.find('option')
        product_details = product_and_related[details_beginning:details_ending]
        related = product_and_related[details_ending:]
        product_info = product_details.split('/n')[1:-1]
        keys = product_info[::2]
        values = product_info[1::2]
        for key, value in zip(keys,values): # add the additional data to the dataframe
            df[key] = value

        scrapped_home_data = scrapped_home_data.append(df, ignore_index=True) # update the global daframe
        hspbar.update(1)
    else:
        df = pd.DataFrame()
        image_element = soup.find_all("img")[2]['src'] # scrap the image link
        df['image'] = image_element # adding the image link to results
        page_text = soup.get_text('/n') # add seperator so you can split later
        relevant_beginning = page_text.find('Main') + 15
        relevant_ending = page_text.find('addGeneral Terms')
        product_and_related = page_text[relevant_beginning:relevant_ending]
        details_beginning = product_and_related.find('Details') + 7
        details_ending  = product_and_related.find('Select')
        product_details = product_and_related[details_beginning:details_ending]
        related = product_and_related[details_ending:]
        product_info = product_details.split('/n')[1:-1]
        keys = product_info[::2]
        values = product_info[1::2]
        for key, value in zip(keys,values): # add the additional data to the dataframe
            df[key] = value

        scrapped_home_data = scrapped_home_data.append(df, ignore_index=True) # update the global daframe
        hspbar.update(1)

In [7]:
def scrap_business(link):
    global scrapped_business_data
    page = requests.get(link)
    soup = BeautifulSoup(page.content, "html.parser")
    html = list(soup.children)[1]
    body = list(html.children)[1]
    if len(str(list(body.children)[-1])) > 48: # check if data available
        item_dict = eval(str(list(body.children)[-1])[35:-9].replace('\n','').replace('@','').replace('[','').replace(']',''))
        df = pd.json_normalize(item_dict, record_prefix='Prefix.') # to be used with single record
        image_element = soup.find_all("img")[2]['src'] # scrap the image link
        df['image'] = image_element # adding the image link to results
        page_text = soup.get_text('/n') # add seperator so you can split later
        relevant_beginning = page_text.find('Main') + 15
        relevant_ending = page_text.find('addGeneral Terms')
        product_and_related = page_text[relevant_beginning:relevant_ending]
        details_beginning = product_and_related.find('Details') + 7
        details_ending  = product_and_related.find('option')
        product_details = product_and_related[details_beginning:details_ending]
        related = product_and_related[details_ending:]
        product_info = product_details.split('/n')[1:-1]
        keys = product_info[::2]
        values = product_info[1::2]
        for key, value in zip(keys,values): # add the additional data to the dataframe
            df[key] = value

        scrapped_business_data = scrapped_business_data.append(df, ignore_index=True) # update the global daframe
        bspbar.update(1)
    else:
        df = pd.DataFrame()
        image_element = soup.find_all("img")[2]['src'] # scrap the image link
        df['image'] = image_element # adding the image link to results
        page_text = soup.get_text('/n') # add seperator so you can split later
        relevant_beginning = page_text.find('Main') + 15
        relevant_ending = page_text.find('addGeneral Terms')
        product_and_related = page_text[relevant_beginning:relevant_ending]
        details_beginning = product_and_related.find('Details') + 7
        details_ending  = product_and_related.find('option')
        product_details = product_and_related[details_beginning:details_ending]
        related = product_and_related[details_ending:]
        product_info = product_details.split('/n')[1:-1]
        keys = product_info[::2]
        values = product_info[1::2]
        for key, value in zip(keys,values): # add the additional data to the dataframe
            df[key] = value

        scrapped_business_data = scrapped_business_data.append(df, ignore_index=True) # update the global daframe
        bspbar.update(1)

In [8]:
if __name__ == "__main__":
    print("[+] Number of Pages to Crawl:", max_urls)
    cpbar = tqdm(total=max_urls, desc="Crawling Progress")
    crawl('https://jumla.io/category/main')
    print("[+] External Links Found:", len(external_urls))
    print("[+] Internal Links Found:", len(internal_urls))
    print("[+] Total:", len(external_urls) + len(internal_urls))
    internal_urls_list = list(internal_urls)
    products_links = [link for link in internal_urls_list if '/item/' in link] # desired links
    not_products_links = [link for link in internal_urls_list if '/item/' not in link] # discarded links
    home_products_links = [link for link in products_links if '/for-home' in link]
    business_products_links = [link for link in products_links if '/for-business' in link] 
    print("[+] Valid Links to Scrap:", len(products_links))
    print("[+] Invalid Links:", len(not_products_links) + len(external_urls))
    hspbar = tqdm(total=len(home_products_links), desc="Scraping Home Products Progress")
    for hlink in home_products_links:
        scrap_home(hlink)
    hspbar.close()
    bspbar = tqdm(total=len(business_products_links), desc="Scraping Business Products Progress")
    for blink in business_products_links:
        scrap_business(blink)
    bspbar.close()

[+] Number of Pages to Crawl: 3


Crawling Progress:   0%|          | 0/3 [00:00<?, ?it/s]

[+] External Links Found: 5
[+] Internal Links Found: 70
[+] Total: 75
[+] Valid Links to Scrap: 57
[+] Invalid Links: 18


Scraping Home Products Progress:   0%|          | 0/15 [00:00<?, ?it/s]

Scraping Business Products Progress:   0%|          | 0/42 [00:00<?, ?it/s]